In [2]:
import numpy as np
import pandas as pd
from pyomo.environ import *

<font size=18>Lesson 12 - Scheduling and Decision Analysis with Uncertainty</font>

# General Questions

## Question 1: Practice fetching values from a triangular distribution. <font color="magenta">(2 points)</font>
Given the triplet (8, 20, 50) that corresponds to the left, mode, and right sides of the triangle, fetch 10 random samples from a triangular distribution. Use a numpy random seed of 121. Use either a loop or a vector approach. 

What is the last number in the draw?

* 18
* 20
* 50
* **43**
* 19
* 26


In [3]:
np.random.seed(121)

tri = np.random.triangular(8, 20, 50, size=10)
tri

array([15.49071007, 18.30664143, 18.83573227, 16.75100319, 35.37209701,
       22.68660517, 26.34193913, 32.09358124, 19.19124108, 43.71596014])

## Question 2: Practice Fetching Values from an Exponential Distribution. <font color="magenta">(2 points)</font>

Using a numpy random seed of 121, fetch 10 values from an exponential distribution with a scale of 100. What is the last number in the draw, rounded to 2 digits?

* **346.28**
* 11.80
* 177.30
* 81.15
* 28.57
* 16.48


In [4]:
np.random.seed(121)

exp = np.random.exponential(100, size=10)
exp

array([ 11.80302477,  23.66944192,  26.5219571 ,  16.48092647,
       177.30052637,  52.41125299,  81.14592057, 136.85485229,
        28.56837445, 346.28408745])

# Reliable Construction Scheduling, with Uncertainty

<font color="red">This intro is not included in Canvas</font>

Reliable Construction wants to determine when each activity should start in a construction schedule in order to minimize the overall time it takes to construct a large commercial building. From past projects, they know that the time each task takes tends to follow a triangular distribution, where their optimistic estimate is the left (low) side of the triangle, their most likely estimate is the middle (mode) of the triangle, and their pessimistic estimate is the right (high) side of the triangle. Use these estimates and the solution to this problem from Lesson 3 to construct and solve a single Pyomo model to determine the optimal schedule that takes the minimal amount of time. 

* Use a numpy random seed of *121* at the beginning of your code cell. (If you write a function, put it inside, but at the beginning of the function.)
* Be sure to use int() or .astype('int') in your code to round down to integers.

**Note: We strongly encourage you to read all the instructions and use functions to modularize your code. Our solutions involve three functions - one to run the simulations (with any number of iterations, and with or without artifacts), one to figure profit, and one to figure probability. If you use a function, be sure to set your seed inside your function, so that it is set each time you call the function.**

| Activity  | Description  | Immediate Predecessors  | Optimistic Estimate  | Most Likely Estimate  |  Pessimistic Estimate |
|---|---|---|---|---|---|
| A  | Excavate  |  -- |   7 |  14 |  21 |
| B  | Lay the Foundation  | A  |14  | 21  | 56  |
| C  |  Put up the rough wall |  B | 42 |  63 |  126 |
| D  |  Put up the roof | C  |  28  | 35  |  70 |
| E  |  Install the exterior plumbing | C  | 7  | 28  |  35 |
| F  |  Install the interior plumbing | E  | 28  | 35  |  70 |
| G  |  Put up the exterior siding | D  | 35  | 42  | 77  |
| H  |  Do the exterior painting | E,G  |35  | 56  | 119  |
| I  |  Do the electrical work | C  |21  | 49  | 63  |
| J  |  Put up the wallboard |  F,I |  21 |  63 |  63 |
| K  |  Install the flooring |  J | 21 |  28 | 28  |
| L  |  Do the interior painting |  J | 19  |  35 | 42  |
| M  |  Install the exterior fixtures | H  | 7 | 14  | 21  |
| N  |  Install the interior fixtures | K,L | 35 | 35  | 63  |

## Question 3: What is the minimum number of days that Reliable Construction will take? <font color="magenta">(2 points)</font>

* 301 days
* 323 days
* 400 days
* **308 days**
* 289 days


In [78]:
# Your code here
#hint - this is how we set up our function
from pyomo.environ import *

def construct_schedule(simSize=1, artifacts=False, debug=False):
	np.random.seed(121)
    #yourcodehere

	pred_dict = {
		'B':['A'],
		'C':['B'],
		'D':['C'],
		'E':['C'],
		'F':['E'],
		'G':['D'],
		'H':['E','G'],
		'I':['C'],
		'J':['F','I'],
		'K':['J'],
		'L':['J'],
		'M':['H'],
		'N':['K','L'],
		'final':['A','B','C','D','E','F','G','H','I','J','K','L','M','N']
	}

	descriptions = ['excavation', 'laying foundation', 'rough wall',
				   'roofing', 'exterior plumbing', 'interior plumbing',
				   'exterior siding', 'exterior painting', 'electrical work', 'wallboard', 'flooring',
				   'interior painting', 'exterior fixtures', 'interior fixtures', 'final']
	tasks = ['A', *pred_dict]
	task_dict = dict(zip(tasks, descriptions))
	all_durations = [(7,14,21), 
			 (14,21,56), 
			 (42,63,126),
			 (28,35,70),
			 (7,28,35),
			 (28,35,70),
			 (35,42,77),
			 (35,56,119),
			 (21,49,63),
			 (21,63,63),
			 (21,28,28),
			 (19,35,42),
			 (7,14,21),
			 (35,35,63),
			 0]
	
	# Track results
	results = []
	
	for s in range(simSize):
		durations = []
		for d in range(len(all_durations)-1):
			left, mode, right = all_durations[d]
			if (d == 0) and (artifacts):
				left, mode, right = (14, 29, 386)
			tri = np.random.triangular(left, mode, right)
			durations.append(int(tri))
		durations.append(all_durations[-1])
	
		task_duration_dict = dict(zip(tasks, durations))

		#Model
		M = ConcreteModel(name = "Schedule")

		M.start = Var(tasks, domain = NonNegativeReals)

		M.length = Objective( expr = M.start['final'], sense = minimize )

		M.pred_ct = ConstraintList()
		for after in pred_dict.keys():
			for before in pred_dict[after]:
				M.pred_ct.add( M.start[after] - task_duration_dict[before] >= M.start[before])

		### Solution ###
		solver = SolverFactory('glpk')
		solver.solve(M)
		
		results.append(M.length())

	# Return as a numpy array of integers
	return(np.array(results).astype(int))


In [79]:
construct_schedule()

array([308])

# Building a Larger Simulation
A single simulation doesn't really tell us much. We really need to run the simulation multiple times in order to get a good estimate of the minimum amount of days it might take. Use a loop or vector approach to run 1000 iterations of the Pyomo model. For each iteration, track the optimal (minimum) number of days it will take to complete the project. You should have a 1000 number numpy array when you are done.

**Hint: The first value in your array should match the value found in Question 3.**

## Question 4: What is the average number of days it will take, rounded down to the nearest integer. <font color="magenta">(2 points)</font>

**Hint: use .astype("int"), not np.round() on the array of results from your simulation.**

* **316**
* 308
* 324
* 313
* 298


In [19]:
results = construct_schedule(1000)
np.mean(results).astype(int)

316

# Considering Profit
<font color="red">Note: This intro not included in Canvas.</font>

Reliable is interested in estimating their total profit on this contract. Their base profit is \$5.4 million dollars. But there is a bonus for completing the project early, and a daily penalty for finishing the project late.

Consider these factors.

* The base amount that Reliable will earn is \$5.4 million.
* If Reliable completes the project in 280 days or less, they will get a bonus of \$150,000 (\$.15 million).
* If Reliable misses the deadline of 329 days, there will be a \$25,000 (\$.025 million) penalty for each day over 329.

Use your results from your 1000 iteration simulation to calculate the average profit.

## Question 5: What is the average profit in millions, rounded to 2 digits? <font color="magenta">(2 points)</font>

* 5.12
* **5.28**
* 6.0
* 5.12
* 5.21

In [69]:
def fetchProfit(totalDays, artifacts=False):
	'''
	Parameters:
	totalDays - a numpy array of the total days it took for the construction project on each iteration of the simulation
	artifacts - a flag to determine if we're simulating artifacts or not
	'''
	base = 5.4
	
	# bonus if project done in 280 days or less
	bonus = .15
	
	# daily penalty for every day over 329 days
	daily_penalty = -.025
	
	profit = np.where(totalDays<=280, base+bonus, totalDays)
	profit = np.where((profit>280) & (profit<=329), base, profit)
	profit = np.where(profit>329, ((profit-329)*daily_penalty)+base, profit)
	
	if artifacts:
		additional_costs = np.random.exponential(scale=.1, size=len(totalDays))
		profit = profit-additional_costs
	
	return(profit)

In [35]:
profits = fetchProfit(results)
np.mean(profits)

5.278175000000001

## Question 6: What is the probability that Reliable Company will finish the bid in less than 280 days? Choose the closest answer. <font color="magenta">(2 points)</font>

* .290
* .643
* .192
* **.067**
* .218

## Question 7: What is the probability that Reliable Company will finish the bid in more than 280 days, but less than or equal to 329 days? Choose the closest answer.<font color="magenta">(2 points)</font>

* .290
* **.643**
* .192
* .067
* .218

## Question 8: What is the probability that Reliable Company will finish the bid in more than 329 days? Choose the closest answer.<font color="magenta">(2 points)</font>

* **.290**
* .643
* .192
* .067
* .218


In [46]:
def findProbs(totalDays):
	simSize = len(totalDays)
	
	bonus_prob = len(np.where(totalDays<=280)[0])/simSize
	base_prob = len(np.where((totalDays>280) & (totalDays<=329))[0])/simSize
	penalty_prob = len(np.where(totalDays>329)[0])/simSize
	
	print(f'Probability of bonus: {bonus_prob}\nProbability of base return: {base_prob}\nProbability of incurring penalty for at least one day: {penalty_prob}')
	
findProbs(results)

Probability of bonus: 0.067
Probability of base return: 0.643
Probability of incurring penalty for at least one day: 0.29


# Add artifacts and include random cost
<font color="red">Note: This intro not included in Canvas</font>

From past experience, we know that special artifacts are sometimes found in the area where Reliable Construction is planning this building project.  When special artifacts are found, the excavation phase takes considerably longer and the entire project costs more - sometimes much more. They're never quite sure how much longer it will take, but it averages around an extra 15 days, and takes at least an extra 7 days. They've seen some sites where relocating the special artifacts took as much as 365 extra days (yes - a whole year)! These extra days follow a triangular distribution. **Hint: the extra days ONLY impact the excavation task**

In addition, there are usually unanticipated costs that include fines and other things.  The accounting departments suggest that we model those costs with an exponential distribution with mean (scale) \\$100,000.

Run a second 1000-iteration simulation in which artifacts are always found.

## Question 9: What is the average minimum number of days (rounded down) the project takes when artifacts are found? <font color="magenta">(2 points)</font>

* 400
* 428
* 475
* **448**
* 399


In [49]:
artifact_results = construct_schedule(1000, artifacts=True)
np.mean(artifact_results).astype(int)

448

## Question 10: What is the average profit in millions when artifacts are found, rounded to 2 digits? <font color="magenta">(2 points)</font>

* **2.29**
* 3.00
* 2.13
* 2.87
* 2.65

In [76]:
artifact_profits = fetchProfit(artifact_results, artifacts=True)
np.mean(artifact_profits)

2.290260059039988

## Question 11: What is the probability that Reliable Company will finish the bid in less than 280 days when artifacts are found? <font color="magenta">(2 points)</font>

* **.002**
* .927
* .320
* .508
* .071

## Question 12: What is the probability that Reliable Company will finish the bid in more than 280 days, but less than or equal to 329 days when artifacts are found? <font color="magenta">(2 points)</font>

* .002
* .927
* .320
* .508
* **.071**

## Question 13: What is the probability that Reliable Company will finish the bid in more than 329 days when artifacts are found? <font color="magenta">(2 points)</font>

* .002
* **.927**
* .320
* .508
* .071



In [77]:
findProbs(artifact_results)

Probability of bonus: 0.002
Probability of base return: 0.071
Probability of incurring penalty for at least one day: 0.927


## Question 14 (Manually graded) <font color="magenta">(2 points)</font>

Clearly dealing with artifacts can be very costly for Reliable Construction.  It is known from past experience that about 30% of building sites in this area contain special artifacts.  Fortunately, they can purchase an insurance policy - a quite expensive insurance policy. The insurance policy costs \$500000 (\$.5 million), but it covers all fines and penalities for delays in the event that special artifacts are found that require remediation. Effectively, this means that Reliable could expect the same profit they would get without insurance (minus the cost of the policy).

Given the estimated profit without artifacts, the estimated profit with artifacts, the cost of insurance, the 30% likelihood of finding artifacts, create a payoff table and determine what decision Reliable should make. Their decision should use as much information as is available.  You should use round the simulated costs to nearest \$100,000 and use units of millions of dollars so that, for example, \$8,675,309 is 8.7 million dollars.

Provide appropriate evidence for the best decision such as a payoff table or picture of a suitable (small) decision tree.




## Question 15: What is the expected payoff? <font color="magenta">(2 points)</font>

* 4.3
* 5.2
* **4.8**
* 5.3
* 2.5

## Question 16: Should Reliable Construction purchase insurance? <font color="magenta">(2 points)</font>

* No
* **Yes**
* It is impossible to tell from this analysis.

# Posterior Probabilities and Full Decision Tree

Reliable has been contacted by an archeological consulting firm. They assess sites and predict whether special artifacts are present. They have a pretty solid track record of being right when artifacts are present - they get it right about 86% of the time. Their track record is less great when they there are no artifacts. They're right about 72% of the time.

Find the posterior probabilities for this decision tree, given the this prior information.

## Question 17: What is the posterior probability of artifacts, given artifacts were predicted? <font color="magenta">(2 points)</font>

* .077
* .432
* **.568**
* .923

## Question 18: What is the posterior probability of no artifacts, given no artifacts were predicted? <font color="magenta">(2 points)</font>

* .077
* .432
* .568
* **.923**

<font color="green">Be prepared to share a decision tree or some other mechanism of determining posterior probabilities, if asked.</font>

## Question 19: Create a Full Decision Tree (Manually graded) <font color="magenta">(6 points)</font>

The consulting fee for the site in question is \$50,000 (\$.05 million). 

Construct a decision tree to help Reliable decide if they should hire the consulting firm or not and if they should buy insurance or not.  Again, you should round the simulated costs to nearest $100,000 and use units of millions of dollars (e.g. 3.8 million dollars) in your decision tree.

Include a picture of the tree exported from Silver Decisions.


## Question 20: What is the best course of action for Reliable? <font color="magenta">(2 points)</font>

* Don't hire firm. Buy insurance.
* Don't hire firm. Don't by insurance.
* Hire firm. Don't buy insurance no matter what they predict.
* Hire firm. Buy insurance no matter what they predict.
* **Hire firm. Buy insurance if they predict artifacts. Don't buy insurance if they don't predict artifacts.**

## Question 21: What is the expected payoff? <font color="magenta">(2 points)</font>

* \$6 million
* **\$4.9 million** (though silver decisions shows $4.66 million, this is the closest)
* \$5 million
* \$2.5 million
* \$4.2 million

## Question 22 (Manually graded) <font color="magenta">(4 points)</font>

How confident are you in your decision? What could you do to increase your confidence? 

